In [1]:
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Embedding, LSTM, SpatialDropout1D, Lambda, Bidirectional, \
    TimeDistributed, GRU, Input, concatenate
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [2]:
st_time = time.time()
X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_val = np.load("X_val.npy")
Y_val = np.load("Y_val.npy")
print("Load data time: %.3f seconds" % (time.time() - st_time), flush=True)

Load data time: 0.260 seconds


In [3]:
sequence_length = X_train.shape[1]
sequence_length

20

In [4]:
w2v_length = X_train.shape[2]
w2v_length

320

In [5]:
output_vec_len = Y_train.shape[2]
output_vec_len

4000

## Краткое описание данных:

X_train, X_val - трехмерные матрицы, размерностью [*, 50, 320], где * это количество предложений, 50 - длина каждого предложения (сколько слов в предложении), 320 - вектор-слово Word2Vec.
То есть был некий текст, в котором было очень много строк. Затем брались все слова из первой строки, если их было меньше чем 50, то после первой строки ставилось символ-слово обозначающее конец строки ("/S"), далее брались все слова второй строки и т.д. пока не набиралось 50 слов. Далее для этих слов брались 50 векторов, размерностью 320 каждый. После этого в тексте осуществлялся переход на новую строку и снова начинался набор 50 слов, и так далее пока не заполнилась матрица.
То есть каждая запись в матрице это некоторое предложение, в котором, возможно, менялся контекст, при переходе на новую строку, но знак окончания предыдущего контекста вводился как доп слово.

Y_train, Y_val - трехмерные матрицы, размерностью [*, 50, 4000], где * это количество предложений (соответствует X_train, Y_train), 50 - длина каждого предложения, 4000 - one-hot вектор, где 3999 нулей и 1 единица на том индексе, который соответствует номеру кластера, которому принадлежит слово, следующее за входным.

Попробую простым языком:
Был взят текст, очень большой на много Гигабайт, на одном языке, допустим английском. Далее в нем были найдены 200 тысяч самых популярных (по количеству раз встреченных) слов, из которых сформирован словарь.  
Далее на этом тексте был натренирован Mikolov's Word2Vec. Это N-мерное (в нашем случае 320 мерное) представление слов, натренированное по большому тексту, где похожие или связанные слова находятся близко друг к другу, а совершенно несвязанные, соотвественно, далеко. Эта word2vec модель позволяет при обращении к ней для любого слова получать вектор для этого слова.  
Отдельно нужно указать что есть два "особых" слова, это "UNK" и "/S".  
"UNK" нужен тогда, когда встретится слово, которого не было в тексте для тренировки, либо число его вхождений было очень мало. Если не ошибаюсь этот вектор состоит целиком из чисел около нуля.  
"/S" это вектор обозначающий конец контекста. Например конец предложения или абзаца/параграфа.  
  
Данные в X_train это как раз предложения, в которых взяты 50 слов подряд, а точнее векторы этих слов. Но так как само собой предложения могут быть и меньше 50 слов, берутся из текста слова подряд. Если встречается слово не из словаря - то вместо него подставляется вектор UNK, если конец предложения - подставляется вектор /S. И так пока не наберется 50 векторов. Из * таких предложений и формируется батч.  
А вот Y_train очень связан с X_train таким образом:  
если считать что слова в X_train были взяты в тексте с первого по пятидесятое [0:50], то слова в Y_train взяты со второго по пятьдесят первое [1:51]. Типо если мы даем модели первое слово, она должна предсказать второе слово. Но так как слов в словаре очень много (200 тысяч), то делать выходной вектор размерностью 200000 это очень затратно по памяти. Поэтому была применена хитрость, - все слова (точнее их вектора) из модели word2vec были предварительно откластеризованы, на 4000 кластеров. Таким образом каждое слово имеет номер кластера, которому оно принадлежит. И значит выходной вектор уже можно делать размерностью 4000, что значительно упрощает задачу, с точки зрения требуемых ресурсов. Таким образом наша модель должна получать на вход слово, и предсказывать номер кластера следующего слова. Модель рекуррентная, то есть должна запоминать историю даваемых ей на вход слов, для улучшения качества прогнозирования последующих слов.

In [6]:
X_input = Input(shape=(sequence_length, w2v_length), name="x_input")
gru_layer1 = GRU(512, return_sequences=True, dropout=0.25, recurrent_dropout=0.25)(X_input)
gru_layer2 = GRU(512, return_sequences=True, dropout=0.25, recurrent_dropout=0.25)(gru_layer1)
y_output = TimeDistributed(Dense(output_vec_len, activation="softmax"), name="y_output")(gru_layer2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
model = Model(inputs=[X_input], outputs=[y_output])
model.compile(loss={"y_output": categorical_crossentropy}, optimizer="adam", metrics=["acc"])

In [8]:
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x_input (InputLayer)         [(None, 20, 320)]         0         
_________________________________________________________________
gru (GRU)                    (None, 20, 512)           1279488   
_________________________________________________________________
gru_1 (GRU)                  (None, 20, 512)           1574400   
_________________________________________________________________
y_output (TimeDistributed)   (None, 20, 4000)          2052000   
Total params: 4,905,888
Trainable params: 4,905,888
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.fit(X_train, Y_train, epochs=100, batch_size=500, verbose=1, validation_data=(X_val, Y_val))

Train on 5000 samples, validate on 500 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
5000/5000 [==============================] - 3s 544us/sample - loss: 6.7502 - acc: 0.1647 - val_loss: 5.6450 - val_acc: 0.0669
Epoch 2/100
5000/5000 [==============================] - 2s 393us/sample - loss: 5.6173 - acc: 0.1501 - val_loss: 5.5660 - val_acc: 0.1824
Epoch 3/100
5000/5000 [==============================] - 2s 390us/sample - loss: 5.5281 - acc: 0.1875 - val_loss: 5.5174 - val_acc: 0.1824
Epoch 4/100
5000/5000 [==============================] - 2s 387us/sample - loss: 5.4861 - acc: 0.1875 - val_loss: 5.4824 - val_acc: 0.1824
Epoch 5/100
5000/5000 [==============================] - 2s 389us/sample - loss: 5.4480 - acc: 0.1876 - val_loss: 5.4444 - val_acc: 0.1824
Epoch 6/100
5000/5000 [==============================] - 2s 389us/sample - loss: 5.4035 - acc: 0.1878 - val_loss: 5.3991 - val_acc: 0.1828
Epoch 7/100
5000/5000 [==

Epoch 57/100
5000/5000 [==============================] - 2s 389us/sample - loss: 3.6986 - acc: 0.2902 - val_loss: 4.9882 - val_acc: 0.2162
Epoch 58/100
5000/5000 [==============================] - 2s 391us/sample - loss: 3.6750 - acc: 0.2929 - val_loss: 4.9989 - val_acc: 0.2187
Epoch 59/100
5000/5000 [==============================] - 2s 389us/sample - loss: 3.6446 - acc: 0.2956 - val_loss: 5.0045 - val_acc: 0.2158
Epoch 60/100
5000/5000 [==============================] - 2s 389us/sample - loss: 3.6201 - acc: 0.2968 - val_loss: 5.0182 - val_acc: 0.2149
Epoch 61/100
5000/5000 [==============================] - 2s 390us/sample - loss: 3.5922 - acc: 0.3026 - val_loss: 5.0269 - val_acc: 0.2146
Epoch 62/100
5000/5000 [==============================] - 2s 390us/sample - loss: 3.5637 - acc: 0.3034 - val_loss: 5.0358 - val_acc: 0.2098
Epoch 63/100
5000/5000 [==============================] - 2s 388us/sample - loss: 3.5386 - acc: 0.3080 - val_loss: 5.0487 - val_acc: 0.2121
Epoch 64/100
5000/50

In [10]:
y_pred = model.predict(X_val[0,:,:].reshape([-1,20,320]))
print(y_pred.shape)
y_pred

(1, 20, 4000)


array([[[4.76523798e-09, 8.32621527e-09, 2.86427344e-08, ...,
         5.84346038e-09, 5.52511701e-05, 1.21187168e-05],
        [9.84920479e-10, 9.66474789e-10, 1.69531711e-09, ...,
         8.25243929e-10, 7.96108725e-05, 5.61855404e-06],
        [1.78859256e-08, 1.58294693e-08, 1.45389931e-08, ...,
         1.21539712e-08, 3.45126115e-04, 5.79337211e-06],
        ...,
        [3.04404075e-08, 2.36916460e-08, 1.21197035e-08, ...,
         2.56806185e-08, 1.14094355e-05, 9.96329982e-06],
        [9.04242070e-10, 1.29022715e-09, 1.28306796e-10, ...,
         1.84364068e-09, 2.07305584e-06, 1.56928493e-07],
        [5.93193328e-10, 8.14227130e-10, 1.06404226e-10, ...,
         8.08849765e-10, 6.38008828e-07, 7.18388549e-08]]], dtype=float32)

In [11]:
y_pred.argmax(axis=2)

array([[1793,  375,  375, 1484, 1484, 3361, 3361, 2745, 3361, 1971, 1484,
        1971,  950, 1971, 1971, 3361, 2069,  572, 1484, 1484]])

In [12]:
y_true = Y_val[0].reshape([-1,20,4000])
print(y_true.shape)
y_true

(1, 20, 4000)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [13]:
y_true.argmax(axis=2)

array([[1793, 3707, 2249,  677,   13, 3620,  696,  611, 2365, 1484,    3,
         261, 2210, 3361, 2978, 1971,  550,  572, 3791, 3361]])

То есть сейчас у меня получилась такая архитектура модели, что мне нужно давать на вход фиксированное количество слов (в данном случае 20), и на выходе я получу 20 предсказаний.
А мне хотелось бы получить такую модель, которой я бы мог скармливать по одному слову, получать одно предсказание. далее скармливать второе слово, и при этом модель помнила, что перед этим у нее было первое слово и снова давала предсказание. Затем скармливал третье слово, а модель помнит, что в истории были предыдущие два слова и так далее.

И дополнительно еще, чтобы можно было реализовать ветвление. То есть предсказав для, например, пяти слов, я хочу попробовать два разных шестых слова, то есть как-то зафиксировать состояние модели, попробовать одно шестое слово, попробовать другое шестое слово, но чтобы в обоих случаях история была одинаковая, опирающаяся на первые пять слов.

И также хотелось бы понять, как для модели, которой буду скармливать по одному слову, указать сколько слов назад она будет помнить.
